In [1]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
from skimage.transform import resize
from skimage.filters import gaussian

from vpt.features.features import *
import vpt.utils.image_processing as ip
import vpt.settings as s

from keras import models

%load_ext autoreload
%autoreload 2

Using Theano backend.


In [2]:
participants = ["p1", "p3", "p4", "p5", "p6"]
base_folder = "data/posture"
hand_folder = "hand_data_M3_rad.3"

size = (120, 96)

In [ ]:
# Load all data

X_lh = np.array([])
y_lh = np.array([])
X_rh = np.array([])
y_rh = np.array([])
filenames = np.array([])

for p in participants:
      
    folder = os.path.join(base_folder, p, hand_folder)
    X_lh = np.concatenate( (X_lh, np.load(os.path.join(folder, "X_lh.npy"))))
    y_lh = np.concatenate( (y_lh, np.load(os.path.join(folder, "y_lh.npy"))))
    X_rh = np.concatenate( (X_rh, np.load(os.path.join(folder, "X_rh.npy"))))
    y_rh = np.concatenate( (y_rh, np.load(os.path.join(folder, "y_rh.npy"))))
    filenames = np.concatenate( (filenames, np.load(os.path.join(folder, "filenames.npy"))))
    
print("LH:", X_lh.shape, y_lh.shape)
print("RH:", X_rh.shape, y_rh.shape)
print("Filenames:", filenames.shape)

In [ ]:
X_lh_ds = np.zeros((len(X_lh), size[0], size[1]))
X_rh_ds = np.zeros((len(X_rh), size[0], size[1]))

for i, (lh, rh) in enumerate(zip(X_lh, X_rh)):
    X_lh_ds[i] = resize(ip.normalize(lh), size, preserve_range=True)
    X_rh_ds[i] = resize(ip.normalize(rh), size, preserve_range=True)
    
X_lh_ds = X_lh_ds[:,:,:, np.newaxis]
X_rh_ds = X_rh_ds[:,:,:, np.newaxis]

In [ ]:
cae_lh = models.load_model("data/cae/hands/lh_20171207_encoder_model.h5")
cae_rh = models.load_model("data/cae/hands/rh_20171207_encoder_model.h5")

In [ ]:
encoded_imgs_lh = cae_lh.predict(X_lh_ds)

In [ ]:
encoded_imgs_rh = cae_lh.predict(X_rh_ds)

In [ ]:
encoded_imgs_lh.shape
encoded_imgs_rh.shape

In [ ]:
encoded_imgs_lh = np.reshape(encoded_imgs_lh, (17622, 15*12*50))

In [ ]:
encoded_imgs_rh = np.reshape(encoded_imgs_rh, (17622, 15*12*50))

In [ ]:
encoded_imgs_lh.shape
encoded_imgs_rh.shape

In [ ]:
np.save("data/posture/all/encoded/encoded_imgs_lh.npy", encoded_imgs_lh)
np.save("data/posture/all/encoded/encoded_imgs_rh.npy", encoded_imgs_rh)

In [3]:
encoded_imgs_lh = np.load("data/posture/all/encoded/encoded_imgs_lh.npy")
encoded_imgs_rh = np.load("data/posture/all/encoded/encoded_imgs_rh.npy")

In [4]:
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score

In [ ]:
X_lh_pca = PCA(n_components="mle", svd_solver="full").fit_transform(encoded_imgs_lh)

In [ ]:
X_lh_pca.shape

In [ ]:
np.save("data/posture/all/encoded/encoded_pca_lh.npy", X_lh_pca)

In [ ]:
from IPython.display import Audio
soundfile ='duetta1.wav'

In [ ]:
Audio(url=soundfile, autoplay=True)

In [ ]:
## find all "static" data so we can ignore for now
r = re.compile('p[\d]s')
# r2 = re.compile('p[\d]d')

# remove p#s data
vmatch = np.vectorize(lambda x:bool(r.search(x)))
rem_static = vmatch(filenames)

X_lh2, y_lh2, filenames2 = X_lh_pca[~rem_static], y_lh[~rem_static], filenames[~rem_static]

# seperate p3 from data
r_p3 = re.compile('/p3/')
vmatch = np.vectorize(lambda x:bool(r_p3.search(x)))
sel = vmatch(filenames2)

#augment data with SMOTE (but first split it so we have some good testing data)
# X_train_lh, X_test_lh, y_train_lh, y_test_lh = train_test_split(X_lh2, y_lh2, test_size=.20)
X_train_lh, X_test_lh, y_train_lh, y_test_lh = X_lh2[~sel], X_lh2[sel], y_lh2[~sel], y_lh2[sel]
X_smote_lh, y_smote_lh = SMOTE(kind='svm').fit_sample(X_train_lh, y_train_lh)

In [ ]:
# Plot Data sets
print("SMOTE:", X_smote_lh.shape, y_smote_lh.shape)
print("TEST:", X_test_lh.shape, y_test_lh.shape)

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.hist(y_smote_lh, bins=3, rwidth=.5)
plt.title("SMOTE Training Labels")

plt.subplot(122)
plt.hist(y_test_lh, bins=3, rwidth=.5)
plt.title("Test data Labels")

In [ ]:
# Using Models selected in previous notebook (see model selection)
C = 100
gamma = .1
kernel = 'rbf'
clf_svm = SVC(C=C, gamma=gamma, kernel=kernel)

In [ ]:
scores = cross_val_score(clf_svm, X_smote_lh, y_smote_lh)

In [ ]:
print("Scores:", scores)
print("Avg Score:", np.average(scores))

In [ ]:
clf_svm.fit(X_smote_lh, y_smote_lh)

In [ ]:
y_true, y_pred = y_test_lh, clf_svm.predict(X_test_lh)
print("Test Set Accuracy Score:", accuracy_score(y_true, y_pred))
print("Confustion Matrix:\n", confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred))